In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# 'dog' or 'cat'
pet_type = 'cat'

shelter_url = "https://berkeleyhumane.org/adopt/adopt-a-"+pet_type
shelter = BeautifulSoup(requests.get(shelter_url).content)
adoptable_pets = shelter.find_all(class_="elementor-post__thumbnail__link")

In [3]:
pet_info_summary = pd.DataFrame(columns=["pet_type",
                                         "name", 
                                         "breed", 
                                         "sex",
                                         "age",
                                         "energy_level",
                                         "interests",
                                         "about",
                                         "url"])
    
for pet in adoptable_pets:
    pet_page_url = pet.get("href")
    pet_page = BeautifulSoup(requests.get(pet_page_url).content)
    name = pet_page.title.get_text().strip().split(":")[0]
    
    pet_page_info = pet_page.find_all("section", attrs={"data-id": "17ce8d5"})[0]
    stats = pet_page_info.find_all("h6")
    
    about = pet_page_info.find("div",attrs={"data-id":"ab992b2"}).find_all("p")
    description = ''
    for p in about:
        description += p.get_text().strip()
    
    breed = stats[0].next_sibling.next_sibling.get_text().strip()
    sex = stats[1].next_sibling.next_sibling.get_text().strip()
    age = stats[2].next_sibling.next_sibling.get_text().strip()
    energy = stats[3].next_sibling.next_sibling.get_text().strip()
    interests = stats[4].next_sibling.next_sibling.get_text().strip()
    url = str(pet_page_url)

    info = [pet_type, name, breed, sex, age, energy, interests, description, url]
    
    pet_info_summary.loc[len(pet_info_summary)] = info

pet_info_summary.to_csv("berkeleyhumane_available_"+pet_type+ "s_info.csv", index=False)
pet_info_summary.head()

,pet_type,name,breed,sex,age,energy_level,interests,about,url
0,cat,Nora,Domestic Shorthair,"Neutered Male(s), Spayed Female(s)",2.5 months,Kitten-level playful,"Napping, Being pet, Toys, Puzzle games, Being ...","Hi, we’re Chili (male, brown tabby w/white), S...",https://berkeleyhumane.org/adoptable_cats/chil...
1,cat,Doug,Domestic Shorthair,Neutered Male,11 Years,Low,"Napping, Being pet, Window watching, Being an ...","Hi, I’m Doug!I’m a cuddly lovebug hoping to sp...",https://berkeleyhumane.org/adoptable_cats/doug/
2,cat,Flareon & Sylveon,Domestic Shorthair,"Neutered Male(s), Spayed Female(s)",2.5 months,Kitten-level playful,"Napping, Being pet, Toys, Puzzle games, Being ...","Greetings, we’re Espeon (male, black & white, ...",https://berkeleyhumane.org/adoptable_cats/espe...
3,cat,Sesame,Domestic Shorthair,Spayed Female,3 months,Kitten-level playful,"Napping, Being pet, Toys, Window watching, Bei...","Hello, we’re Matcha, Red Bean, Sesame, Yuzu (m...",https://berkeleyhumane.org/adoptable_cats/matc...
4,cat,Mars,Domestic Shorthair,Neutered Male,1.5 years,Medium,"Napping, Being pet, Window watching, Puzzle ga...","Well hello, I’m Mars!I am a darling and reserv...",https://berkeleyhumane.org/adoptable_cats/mari...
